In [49]:
import os
import json

from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from  langchain.output_parsers import PydanticOutputParser
from typing import List, Dict

from langchain.schema.runnable import RunnableBranch

In [2]:
project_name = "web-order-assistant" # Update with your project name
os.environ["LANGCHAIN_PROJECT"] = project_name # Optional: "default" is used if not set

In [7]:
# ts
os.environ

environ{'ALLUSERSPROFILE': 'C:\\ProgramData',
        'ANDROID_SDK_HOME': 'Android',
        'APPDATA': 'C:\\Users\\ktsfr\\AppData\\Roaming',
        'APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL': 'true',
        'CHOCOLATEYINSTALL': 'C:\\ProgramData\\chocolatey',
        'CHOCOLATEYLASTPATHUPDATE': '133447822579870606',
        'CHOCOLATEYTOOLSLOCATION': 'C:\\tools',
        'CHROME_CRASHPAD_PIPE_NAME': '\\\\.\\pipe\\crashpad_15400_DNBGHWSZNRRSZEWB',
        'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
        'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
        'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
        'COMPUTERNAME': 'DESKTOP-2BOJ8M8',
        'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe',
        'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData',
        'ELECTRON_RUN_AS_NODE': '1',
        'FPS_BROWSER_APP_PROFILE_STRING': 'Internet Explorer',
        'FPS_BROWSER_USER_PROFILE_STRING': 'Default',
        'HOMEDRIVE': 

In [3]:
MODEL = "gpt-3.5-turbo-0613"

# 1

In [4]:
prompt = PromptTemplate.from_template(
    """
    너는 매우 경험이 많고 유능한 주문봇이야
    가게에서 판매하는 상품 정보를 바탕으로 고객 문의에 적절히 답변해주면 돼.
    
    가게에서 판매하는 상품 목록.
    1. 상품: 떡케익5호
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 54,000원
    2. 상품: 무지개 백설기 케익
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 51,500원
    3. 상품: 미니 백설기
    기본 판매 단위: 1세트(35개)
    기본 판매 단위의 가격: 31,500원
    4. 상품: 개별 모듬팩
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 13,500원

    고객 문의는 크게 두 유형으로 구분돼.
    - 단순 문의:단순 문의란 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하지 않는 메시지를 의미해. 
    - 요청 문의: 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하는 메시지를 의미해.

    ##주의할 점##
    - 특정 표현이 포함돼 있는지만을 따져서 단순 문의와 요청 문의를 분류하면 실수할 수 있음. 예를 들어 다음과 같은 상황이 존재할 수 있어. 고객 메시지에 '주문'이란 단어가 포함돼 있어도 주문을 직접 요청하는 게 아니라 주문에 관한 일반적인 질문이나 이야기하는 상황.

    고객 문의를 꼼꼼하게 살펴보고 아래 정보들을 추출하여 json 형태로 추출 결과를 반환해줘. 
    json의 key와 value에 대한 설명:
    user (str): 고객의 사용자 이름
    response_type(str): [general], [response_to_request] 중 하나로 분류. 단순 문의에 대한 응답이라면  [general], 요청 문의에 대한 응답이라면 [response_to_request]
    response(str): 고객 문의에 대한 모델의 답변
    request_type (str): [order], [order_inquiry], [order_change], [order_cancel] 중 하나로 분류
    products (List[Dict]): 주문된 상품들의 목록, 각 상품은 다음 정보를 포함해야 함:
    - name (str): 상품명
    - price (int): 상품 가격
    - quantity (int): 주문 수량
    order_date (str): 문의 요청 날짜와 시간, '2023. 12. 15. 오후 5:54:51'와 같은 형식의 문자열

    출력 결과 예시
    ##단순 문의에 대한 답변 예시##
    {{
    "user": "사용자이름",
    "response_type": "general",
    "response": "사용자 문의에 대한 모델의 답변",
    "request_type": "general_inquiry",
    "products": [],
    "order_date": "2023. 12. 24. 오후 7:52:11"
    }}

    ##요청 문의에 대한 답변 예시##
    {{
    "user": "사용자이름",
    "response_type": "response_to_request",
    "response": "사용자 문의에 대한 모델의 답변",
    "request_type": "order",
    "products": [
        {{
        "name": "상품명1",
        "price": 22000,
        "quantity": 2
        }},
        {{
        "name": "상품명2",
        "price": 30000,
        "quantity": 2
        }}
    ],
    "order_date": "2023. 12. 15. 오후 5:54:51"
    }}
    
    <고객이 현재 입력한 메시지>
    {customer_message}
    <문의 요청 시간>
    {date}
    """
    )

In [50]:
class JsonParser(BaseModel):
    user: str = Field(description="고객의 사용자 이름")
    response_type: str = Field(description="[general], [response_to_request] 중 하나로 분류. 단순 문의에 대한 응답이라면  [general], 요청 문의에 대한 응답이라면 [response_to_request]")
    response : str = Field(description="고객 문의에 대한 모델의 답변")
    request_type: str = Field(description="[order], [order_inquiry], [order_change], [order_cancel] 중 하나로 분류")
    products: List[Dict] = Field(description="주문된 상품들의 목록. 고객 문의가 요청 문의인 경우에만 값을 할당. 상품은 다음 정보를 포함해야 함: name (str): 상품명, price (int): 상품 가격, quantity (int): 주문 수량")
    order_date: str = Field(description="문의 요청 날짜와 시간, '2023. 12. 15. 오후 5:54:51'와 같은 형식의 문자열")

parser = PydanticOutputParser(pydantic_object=JsonParser)

In [6]:
message = "떡케익 5호 2개, 미니 백설기 1개로 주문 좀 변경할게요."
date = "2023. 12. 18. 오후 8:03:18"

In [7]:
chain = prompt | ChatOpenAI(model=MODEL)
# chain = prompt | ChatOpenAI(model=MODEL) | parser
response = chain.invoke({"customer_message": message,"date": date})

print(type(response.content))
print('-'*77)
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
-----------------------------------------------------------------------------
content='{\n"user": "사용자이름",\n"response_type": "response_to_request",\n"response": "주문 변경 요청에 대한 답변입니다.",\n"request_type": "order_change",\n"products": [\n    {\n    "name": "떡케익5호",\n    "price": 54000,\n    "quantity": 2\n    },\n    {\n    "name": "미니 백설기",\n    "price": 31500,\n    "quantity": 1\n    }\n],\n"order_date": "2023. 12. 18. 오후 8:03:18"\n}'


In [9]:
print(type(response.content))
print(response.content)

<class 'str'>
{
"user": "사용자이름",
"response_type": "response_to_request",
"response": "주문 변경 요청에 대한 답변입니다.",
"request_type": "order_change",
"products": [
    {
    "name": "떡케익5호",
    "price": 54000,
    "quantity": 2
    },
    {
    "name": "미니 백설기",
    "price": 31500,
    "quantity": 1
    }
],
"order_date": "2023. 12. 18. 오후 8:03:18"
}


# 2. partial_variables, parser.parser 사용

In [51]:
template = """
    너는 매우 경험이 많고 유능한 주문봇이야
    가게에서 판매하는 상품 정보를 바탕으로 고객 문의에 적절히 답변해주면 돼.
    
    가게에서 판매하는 상품 목록.
    1. 상품: 떡케익5호
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 54,000원
    2. 상품: 무지개 백설기 케익
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 51,500원
    3. 상품: 미니 백설기
    기본 판매 단위: 1세트(35개)
    기본 판매 단위의 가격: 31,500원
    4. 상품: 개별 모듬팩
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 13,500원

    고객 문의는 크게 두 유형으로 구분돼.
    - 단순 문의:단순 문의란 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하지 않는 메시지를 의미해. 
    - 요청 문의: 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하는 메시지를 의미해.

    ##주의할 점##
    - 특정 표현이 포함돼 있는지만을 따져서 단순 문의와 요청 문의를 분류하면 실수할 수 있음. 예를 들어 다음과 같은 상황이 존재할 수 있어. 고객 메시지에 '주문'이란 단어가 포함돼 있어도 주문을 직접 요청하는 게 아니라 주문에 관한 일반적인 질문이나 이야기하는 상황.

    고객 문의를 꼼꼼하게 살펴보고 아래 정보들을 추출하여 json 형태로 추출 결과를 반환해줘. 
    {format_instructions}

    출력 결과 예시
    ##단순 문의에 대한 답변 예시##
    {{
    "user": "사용자이름",
    "response_type": "general",
    "response": "사용자 문의에 대한 모델의 답변",
    "request_type": "general_inquiry",
    "products": [],
    "order_date": "2023. 12. 24. 오후 7:52:11"
    }}

    ##요청 문의에 대한 답변 예시##
    {{
    "user": "사용자이름",
    "response_type": "response_to_request",
    "response": "사용자 문의에 대한 모델의 답변",
    "request_type": "order",
    "products": [
        {{
        "name": "상품명1",
        "price": 22000,
        "quantity": 2
        }},
        {{
        "name": "상품명2",
        "price": 30000,
        "quantity": 2
        }}
    ],
    "order_date": "2023. 12. 15. 오후 5:54:51"
    }}
    
    <고객이 현재 입력한 메시지>
    {customer_message}
    <문의 요청 시간>
    {date}
    """

In [16]:
prompt = PromptTemplate(
    template=template,
    input_variables=["customer_message", "date"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt

PromptTemplate(input_variables=['customer_message', 'date'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"user": {"title": "User", "description": "\\uace0\\uac1d\\uc758 \\uc0ac\\uc6a9\\uc790 \\uc774\\ub984", "type": "string"}, "response_type": {"title": "Response Type", "description": "[general], [response_to_request] \\uc911 \\ud558\\ub098\\ub85c \\ubd84\\ub958. \\ub2e8\\uc21c \\ubb38\\uc758\\uc5d0 \\ub300\\ud55c \\uc751\\ub2f5\\uc774\\ub77c\\uba74  [general], \\uc694\\uccad \\ubb38\\uc758\\uc5d0 \\ub300\\ud55c \\uc751\\

In [19]:
print(prompt.format_prompt(customer_message=message, date=date).text)


    너는 매우 경험이 많고 유능한 주문봇이야
    가게에서 판매하는 상품 정보를 바탕으로 고객 문의에 적절히 답변해주면 돼.
    
    가게에서 판매하는 상품 목록.
    1. 상품: 떡케익5호
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 54,000원
    2. 상품: 무지개 백설기 케익
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 51,500원
    3. 상품: 미니 백설기
    기본 판매 단위: 1세트(35개)
    기본 판매 단위의 가격: 31,500원
    4. 상품: 개별 모듬팩
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 13,500원

    고객 문의는 크게 두 유형으로 구분돼.
    - 단순 문의:단순 문의란 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하지 않는 메시지를 의미해. 
    - 요청 문의: 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하는 메시지를 의미해.

    ##주의할 점##
    - 특정 표현이 포함돼 있는지만을 따져서 단순 문의와 요청 문의를 분류하면 실수할 수 있음. 예를 들어 다음과 같은 상황이 존재할 수 있어. 고객 메시지에 '주문'이란 단어가 포함돼 있어도 주문을 직접 요청하는 게 아니라 주문에 관한 일반적인 질문이나 이야기하는 상황.

    고객 문의를 꼼꼼하게 살펴보고 아래 정보들을 추출하여 json 형태로 추출 결과를 반환해줘. 
    The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "requ

In [24]:
print(type(prompt.format_prompt(customer_message=message, date=date)))
print(type(prompt.format_prompt(customer_message=message, date=date).to_string()))
prompt.format_prompt(customer_message=message, date=date).to_string()

<class 'langchain_core.prompt_values.StringPromptValue'>
<class 'str'>


'\n    너는 매우 경험이 많고 유능한 주문봇이야\n    가게에서 판매하는 상품 정보를 바탕으로 고객 문의에 적절히 답변해주면 돼.\n    \n    가게에서 판매하는 상품 목록.\n    1. 상품: 떡케익5호\n    기본 판매 단위: 1개\n    기본 판매 단위의 가격: 54,000원\n    2. 상품: 무지개 백설기 케익\n    기본 판매 단위: 1개\n    기본 판매 단위의 가격: 51,500원\n    3. 상품: 미니 백설기\n    기본 판매 단위: 1세트(35개)\n    기본 판매 단위의 가격: 31,500원\n    4. 상품: 개별 모듬팩\n    기본 판매 단위: 1개\n    기본 판매 단위의 가격: 13,500원\n\n    고객 문의는 크게 두 유형으로 구분돼.\n    - 단순 문의:단순 문의란 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하지 않는 메시지를 의미해. \n    - 요청 문의: 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하는 메시지를 의미해.\n\n    ##주의할 점##\n    - 특정 표현이 포함돼 있는지만을 따져서 단순 문의와 요청 문의를 분류하면 실수할 수 있음. 예를 들어 다음과 같은 상황이 존재할 수 있어. 고객 메시지에 \'주문\'이란 단어가 포함돼 있어도 주문을 직접 요청하는 게 아니라 주문에 관한 일반적인 질문이나 이야기하는 상황.\n\n    고객 문의를 꼼꼼하게 살펴보고 아래 정보들을 추출하여 json 형태로 추출 결과를 반환해줘. \n    The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "item

In [29]:
prompt.format_prompt(customer_message=message, date=date).to_string()

'\n    너는 매우 경험이 많고 유능한 주문봇이야\n    가게에서 판매하는 상품 정보를 바탕으로 고객 문의에 적절히 답변해주면 돼.\n    \n    가게에서 판매하는 상품 목록.\n    1. 상품: 떡케익5호\n    기본 판매 단위: 1개\n    기본 판매 단위의 가격: 54,000원\n    2. 상품: 무지개 백설기 케익\n    기본 판매 단위: 1개\n    기본 판매 단위의 가격: 51,500원\n    3. 상품: 미니 백설기\n    기본 판매 단위: 1세트(35개)\n    기본 판매 단위의 가격: 31,500원\n    4. 상품: 개별 모듬팩\n    기본 판매 단위: 1개\n    기본 판매 단위의 가격: 13,500원\n\n    고객 문의는 크게 두 유형으로 구분돼.\n    - 단순 문의:단순 문의란 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하지 않는 메시지를 의미해. \n    - 요청 문의: 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하는 메시지를 의미해.\n\n    ##주의할 점##\n    - 특정 표현이 포함돼 있는지만을 따져서 단순 문의와 요청 문의를 분류하면 실수할 수 있음. 예를 들어 다음과 같은 상황이 존재할 수 있어. 고객 메시지에 \'주문\'이란 단어가 포함돼 있어도 주문을 직접 요청하는 게 아니라 주문에 관한 일반적인 질문이나 이야기하는 상황.\n\n    고객 문의를 꼼꼼하게 살펴보고 아래 정보들을 추출하여 json 형태로 추출 결과를 반환해줘. \n    The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "item

In [30]:
chat = ChatOpenAI(model=MODEL)
response = chat(prompt.format_prompt(customer_message=message, date=date).to_string())
print(type)
response 

TypeError: Got unknown type 


In [34]:
model = OpenAI(model_name=MODEL)
response = model(prompt.format_prompt(customer_message=message, date=date).to_string())
print(type)
response 

c:\Users\ktsfr\AppData\Local\pypoetry\Cache\virtualenvs\order-assistant-webservice-xcVC9Bk9-py3.10\lib\site-packages\langchain_community\llms\openai.py:248: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\ktsfr\AppData\Local\pypoetry\Cache\virtualenvs\order-assistant-webservice-xcVC9Bk9-py3.10\lib\site-packages\langchain_community\llms\openai.py:1057: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


<class 'type'>


'{\n    "user": "사용자이름",\n    "response_type": "response_to_request",\n    "response": "고객님의 요청에 따라 주문을 변경했습니다.",\n    "request_type": "order_change",\n    "products": [\n        {\n            "name": "떡케익5호",\n            "price": 54000,\n            "quantity": 2\n        },\n        {\n            "name": "미니 백설기",\n            "price": 31500,\n            "quantity": 1\n        }\n    ],\n    "order_date": "2023. 12. 18. 오후 8:03:18"\n}'

In [36]:
type(response)
parser.parse(response)

JsonParser(user='사용자이름', response_type='response_to_request', response='고객님의 요청에 따라 주문을 변경했습니다.', request_type='order_change', products=[{'name': '떡케익5호', 'price': 54000, 'quantity': 2}, {'name': '미니 백설기', 'price': 31500, 'quantity': 1}], order_date='2023. 12. 18. 오후 8:03:18')

In [39]:
prompt

PromptTemplate(input_variables=['customer_message', 'date'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"user": {"title": "User", "description": "\\uace0\\uac1d\\uc758 \\uc0ac\\uc6a9\\uc790 \\uc774\\ub984", "type": "string"}, "response_type": {"title": "Response Type", "description": "[general], [response_to_request] \\uc911 \\ud558\\ub098\\ub85c \\ubd84\\ub958. \\ub2e8\\uc21c \\ubb38\\uc758\\uc5d0 \\ub300\\ud55c \\uc751\\ub2f5\\uc774\\ub77c\\uba74  [general], \\uc694\\uccad \\ubb38\\uc758\\uc5d0 \\ub300\\ud55c \\uc751\\

In [40]:
chain = prompt | chat
response = chain.invoke({"customer_message": message,"date": date})

print(type(response))
response 

<class 'langchain_core.messages.ai.AIMessage'>


AIMessage(content='{\n  "user": "사용자이름",\n  "response_type": "response_to_request",\n  "response": "고객님의 주문 변경 요청에 대한 답변입니다. 변경된 주문 내역은 아래와 같습니다.",\n  "request_type": "order_change",\n  "products": [\n    {\n      "name": "떡케익5호",\n      "price": 54000,\n      "quantity": 2\n    },\n    {\n      "name": "미니 백설기",\n      "price": 31500,\n      "quantity": 1\n    }\n  ],\n  "order_date": "2023. 12. 18. 오후 8:03:18"\n}')

In [42]:
parser.parse(response.content)

JsonParser(user='사용자이름', response_type='response_to_request', response='고객님의 주문 변경 요청에 대한 답변입니다. 변경된 주문 내역은 아래와 같습니다.', request_type='order_change', products=[{'name': '떡케익5호', 'price': 54000, 'quantity': 2}, {'name': '미니 백설기', 'price': 31500, 'quantity': 1}], order_date='2023. 12. 18. 오후 8:03:18')

In [52]:
message = "판매 상품 좀 알 수 있을까요?"

chain = prompt | chat
response = chain.invoke({"customer_message": message,"date": date})

print(type(response.content))
response.content 

<class 'str'>


'{\n  "user": "사용자이름",\n  "response_type": "general",\n  "response": "가게에서 판매하는 상품 목록은 다음과 같습니다.\\n\\n1. 상품: 떡케익5호\\n기본 판매 단위: 1개\\n기본 판매 단위의 가격: 54,000원\\n\\n2. 상품: 무지개 백설기 케익\\n기본 판매 단위: 1개\\n기본 판매 단위의 가격: 51,500원\\n\\n3. 상품: 미니 백설기\\n기본 판매 단위: 1세트(35개)\\n기본 판매 단위의 가격: 31,500원\\n\\n4. 상품: 개별 모듬팩\\n기본 판매 단위: 1개\\n기본 판매 단위의 가격: 13,500원",\n  "request_type": "general_inquiry",\n  "products": [],\n  "order_date": "2023. 12. 18. 오후 8:03:18"\n}'

In [48]:
intention_classifier_prompt = PromptTemplate.from_template(
    """
    너는 고객 메시지의 의도를 정확하게 분류하는 주문봇이야.
    너가 분류해야 하는 의도는 아래 두 가지야. 
    - 일반 문의: 고객이 특정 작업과 관련된 질문, 응답 등을 하는 경우. 특정 작업과 관련된 말이지만 실제로 특정 작업을 요청하지 않는 모든 경우가 여기에 해당해.
    - 요청 문의: 고객이 지금 실제로 특정 작업을 요청 하는 경우. 작업 요청 종류는 주문 요청, 주문 조회 요청, 주문 변경 요청, 주문 취소 요청 중 하나야. 주문 조회, 주문 변경, 주문 취소는 <이전>대화에 고객이 현재 말하는 주문 내역이 포함돼 있어야만 해. 
    '일반 문의'와 '요청 문의'의 핵심적인 차이는 특정 작업을 지금 요청하고 있는가 아닌가야.
    특정 작업에 관한 메시지더라도, 현재 그 작업을 명시적으로 요청하지 않는다면 '일반 문의'로 분류해야 해
    
    분류 결과는 일반 문의와 요청 문의 중 하나여야만 해.
    
    <고객이 현재 입력한 메시지>
    {customer_message}
    분류 결과:
    """
    )

intention_classifier_chain = intention_classifier_prompt | chat

message = "주문 내역 좀 알고 싶어요"
response = intention_classifier_chain.invoke({"customer_message": message})
response.content


'요청 문의'

In [ ]:


chat_type_branch = RunnableBranch(
  (lambda x: x["intention"] == "일반 문의", response_chain),
  handle_request_chain
)

# 3 하나의 chain만 사용하는 방향으로 일단 결정

In [53]:
template = """
    너는 매우 경험이 많고 유능한 주문봇이야
    가게에서 판매하는 상품 정보를 바탕으로 고객 문의에 적절히 답변해주면 돼.
    
    가게에서 판매하는 상품 목록.
    1. 상품: 떡케익5호
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 54,000원
    2. 상품: 무지개 백설기 케익
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 51,500원
    3. 상품: 미니 백설기
    기본 판매 단위: 1세트(35개)
    기본 판매 단위의 가격: 31,500원
    4. 상품: 개별 모듬팩
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 13,500원

    고객 문의는 크게 두 유형으로 구분돼.
    - 단순 문의:단순 문의란 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하지 않는 메시지를 의미해. 
    - 요청 문의: 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하는 메시지를 의미해.

    ##주의할 점##
    - 특정 표현이 포함돼 있는지만을 따져서 단순 문의와 요청 문의를 분류하면 실수할 수 있음. 예를 들어 다음과 같은 상황이 존재할 수 있어. 고객 메시지에 '주문'이란 단어가 포함돼 있어도 주문을 직접 요청하는 게 아니라 주문에 관한 일반적인 질문이나 이야기하는 상황.

    고객 문의를 꼼꼼하게 살펴보고 아래 정보들을 추출하여 json 형태로 추출 결과를 반환해줘. 
    {format_instructions}

    출력 결과 예시
    ##단순 문의에 대한 답변 예시##
    {{
    "user": "사용자이름",
    "response_type": "general",
    "response": "사용자 문의에 대한 모델의 답변",
    "request_type": "general_inquiry",
    "products": [],
    "order_date": "2023. 12. 24. 오후 7:52:11"
    }}

    ##요청 문의에 대한 답변 예시##
    {{
    "user": "사용자이름",
    "response_type": "response_to_request",
    "response": "사용자 문의에 대한 모델의 답변",
    "request_type": "order",
    "products": [
        {{
        "name": "상품명1",
        "price": 22000,
        "quantity": 2
        }},
        {{
        "name": "상품명2",
        "price": 30000,
        "quantity": 2
        }}
    ],
    "order_date": "2023. 12. 15. 오후 5:54:51"
    }}
    
    <고객이 현재 입력한 메시지>
    {customer_message}
    <문의 요청 시간>
    {date}
    """

products에 값 할당 조건 추가

In [54]:
class JsonParser(BaseModel):
    user: str = Field(description="고객의 사용자 이름")
    response_type: str = Field(description="[general], [response_to_request] 중 하나로 분류. 단순 문의에 대한 응답이라면  [general], 요청 문의에 대한 응답이라면 [response_to_request]")
    response : str = Field(description="고객 문의에 대한 모델의 답변")
    request_type: str = Field(description="[order], [order_inquiry], [order_change], [order_cancel] 중 하나로 분류")
    products: List[Dict] = Field(description="주문된 상품들의 목록. 고객 문의가 요청 문의인 경우에만 값을 할당. 상품은 다음 정보를 포함해야 함: name (str): 상품명, price (int): 상품 가격, quantity (int): 주문 수량")
    order_date: str = Field(description="문의 요청 날짜와 시간, '2023. 12. 15. 오후 5:54:51'와 같은 형식의 문자열")

parser = PydanticOutputParser(pydantic_object=JsonParser)

In [55]:
message = "판매 상품 좀 알 수 있을까요?"

chain = prompt | chat | parser
response = chain.invoke({"customer_message": message,"date": date})

print(type(response))
response

<class '__main__.JsonParser'>


JsonParser(user='고객이름', response_type='general', response='가게에서 판매하는 상품 목록은 다음과 같습니다:\n1. 상품: 떡케익5호\n   기본 판매 단위: 1개\n   기본 판매 단위의 가격: 54,000원\n2. 상품: 무지개 백설기 케익\n   기본 판매 단위: 1개\n   기본 판매 단위의 가격: 51,500원\n3. 상품: 미니 백설기\n   기본 판매 단위: 1세트(35개)\n   기본 판매 단위의 가격: 31,500원\n4. 상품: 개별 모듬팩\n   기본 판매 단위: 1개\n   기본 판매 단위의 가격: 13,500원', request_type='general_inquiry', products=[], order_date='2023. 12. 18. 오후 8:03:18')

In [57]:
response.response_type

'general'

In [58]:
response.get("response_type")

AttributeError: 'JsonParser' object has no attribute 'get'

In [60]:
class JsonParser(BaseModel):
    user: str = Field(description="고객 아이디")
    response_type: str = Field(description="[general], [response_to_request] 중 하나로 분류. 단순 문의에 대한 응답이라면  [general], 요청 문의에 대한 응답이라면 [response_to_request]")
    response : str = Field(description="고객 문의에 대한 모델의 답변")
    request_type: str = Field(description="[order], [order_inquiry], [order_change], [order_cancel] 중 하나로 분류")
    products: List[Dict] = Field(description="주문된 상품들의 목록. 고객 문의가 요청 문의인 경우에만 값을 할당. 상품은 다음 정보를 포함해야 함: name (str): 상품명, price (int): 상품 가격, quantity (int): 주문 수량")
    order_date: str = Field(description="문의 요청 날짜와 시간을 ISO 형식으로 추출")
    
parser = PydanticOutputParser(pydantic_object=JsonParser)

In [62]:
date

'2023. 12. 18. 오후 8:03:18'

In [61]:
chain = prompt | chat | parser

message = "판매 상품 좀 알 수 있을까요?"

chain = prompt | chat | parser
response = chain.invoke({"customer_message": message,"date": date})

response

JsonParser(user='고객이름', response_type='response_to_request', response='가게에서 판매하는 상품 정보입니다.', request_type='order_inquiry', products=[{'name': '떡케익5호', 'price': 54000, 'quantity': 1}, {'name': '무지개 백설기 케익', 'price': 51500, 'quantity': 1}, {'name': '미니 백설기', 'price': 31500, 'quantity': 1}, {'name': '개별 모듬팩', 'price': 13500, 'quantity': 1}], order_date='2023. 12. 18. 오후 8:03:18')

In [72]:
template = """
    너는 매우 경험이 많고 유능한 주문봇이야
    가게에서 판매하는 상품 정보를 바탕으로 고객 문의에 적절히 답변해주면 돼.
    
    가게에서 판매하는 상품 목록.
    1. 상품: 떡케익5호
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 54,000원
    2. 상품: 무지개 백설기 케익
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 51,500원
    3. 상품: 미니 백설기
    기본 판매 단위: 1세트(35개)
    기본 판매 단위의 가격: 31,500원
    4. 상품: 개별 모듬팩
    기본 판매 단위: 1개
    기본 판매 단위의 가격: 13,500원

    고객 문의는 크게 두 유형으로 구분돼.
    - 단순 문의:단순 문의란 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하지 않는 메시지를 의미해. 
    - 요청 문의: 주문 요청, 주문내역 조회 요청, 주문취소 요청 등 어떤 작업을 요청하는 메시지를 의미해.

    ##주의할 점##
    - 특정 표현이 포함돼 있는지만을 따져서 단순 문의와 요청 문의를 분류하면 실수할 수 있음. 예를 들어 다음과 같은 상황이 존재할 수 있어. 고객 메시지에 '주문'이란 단어가 포함돼 있어도 주문을 직접 요청하는 게 아니라 주문에 관한 일반적인 질문이나 이야기하는 상황.

    고객 문의를 꼼꼼하게 살펴보고 아래 정보들을 추출하여 json 형태로 추출 결과를 반환해줘. 
    {format_instructions}
    
    <고객이 현재 입력한 메시지>
    {customer_message}
    <문의 요청 시간>
    {date}
    """

In [76]:
prompt = PromptTemplate(
    template=template,
    input_variables=["customer_message", "date"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt

PromptTemplate(input_variables=['customer_message', 'date'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"user": {"title": "User", "description": "\\uace0\\uac1d \\uc544\\uc774\\ub514", "type": "string"}, "response_type": {"title": "Response Type", "description": "[general], [response_to_request] \\uc911 \\ud558\\ub098\\ub85c \\ubd84\\ub958. \\ub2e8\\uc21c \\ubb38\\uc758\\uc5d0 \\ub300\\ud55c \\uc751\\ub2f5\\uc774\\ub77c\\uba74  [general], \\uc694\\uccad \\ubb38\\uc758\\uc5d0 \\ub300\\ud55c \\uc751\\ub2f5\\uc774\\ub77c\\u

In [73]:
class JsonParser(BaseModel):
    user: str = Field(description="고객 아이디")
    response_type: str = Field(description="[general], [response_to_request] 중 하나로 분류. 단순 문의에 대한 응답이라면  [general], 요청 문의에 대한 응답이라면 [response_to_request]")
    response : str = Field(description="고객 문의에 대한 모델의 답변")
    request_type: str = Field(description="[order], [order_inquiry], [order_change], [order_cancel] 중 하나로 분류")
    products: List[Dict] = Field(description="주문된 상품들의 목록. 고객 문의가 요청 문의인 경우에만 값을 할당. 상품은 다음 정보를 포함해야 함: name (str): 상품명, price (int): 상품 가격, quantity (int): 주문 수량")
    order_date: str = Field(description="문의 요청 날짜와 시간. 형식 예시: Wed Dec 20 2023 16:47:27 GMT+0900")
    
parser = PydanticOutputParser(pydantic_object=JsonParser)

In [74]:
date = "Wed Dec 20 2023 16:47:27 GMT+0900"

In [77]:
chain = prompt | chat | parser

message = "판매 상품 좀 알 수 있을까요????"

chain = prompt | chat | parser
response = chain.invoke({"customer_message": message,"date": date})

response

JsonParser(user='판매 상품 좀 알 수 있을까요????', response_type='general', response='가게에서 판매하는 상품 목록은 다음과 같습니다.\n1. 상품: 떡케익5호\n   기본 판매 단위: 1개\n   기본 판매 단위의 가격: 54,000원\n2. 상품: 무지개 백설기 케익\n   기본 판매 단위: 1개\n   기본 판매 단위의 가격: 51,500원\n3. 상품: 미니 백설기\n   기본 판매 단위: 1세트(35개)\n   기본 판매 단위의 가격: 31,500원\n4. 상품: 개별 모듬팩\n   기본 판매 단위: 1개\n   기본 판매 단위의 가격: 13,500원', request_type='order_inquiry', products=[{'name': '떡케익5호', 'price': 54000, 'quantity': 1}, {'name': '무지개 백설기 케익', 'price': 51500, 'quantity': 1}, {'name': '미니 백설기', 'price': 31500, 'quantity': 1}, {'name': '개별 모듬팩', 'price': 13500, 'quantity': 1}], order_date='Wed Dec 20 2023 16:47:27 GMT+0900')

In [87]:
response.products[0].get("hi")

In [92]:
print(f"test\n아아 마이크 테스트 중")

test
아아 마이크 테스트 중
